# Enriched Treatment Data Visualization

This notebook visualizes enriched treatment data from any agency (IFPRS, USFS, BLM, NPS, CNRA, CalTrans, Timber Industry, etc.) from the GDB file containing treatment activities across California.

## Configuration

Update the paths below to visualize different enriched datasets:

In [ ]:
# ============================================================
# CONFIGURATION - Update these paths for different datasets
# ============================================================

# Examples of enriched datasets:
# - IFPRS:  "/tmp/IFPRS_2023_2025.gdb", "IFPRS_enriched_20260120"
# - USFS:   "/tmp/USFS_1950_2025.gdb", "USFS_Region04_enriched_20260120"
# - BLM:    "/tmp/BLM_1950_2025.gdb", "BLM_enriched_20260120"
# - NPS:    "/tmp/NPS_1950_2025.gdb", "NPS_enriched_20260120"
# - CNRA:   "/tmp/CNRA_2023_2025.gdb", "CNRA_enriched_20260120"
# - CalTrans: "/tmp/CalTrans_2023_2025.gdb", "CalTrans_enriched_20260120"
# - Timber Industry: "/tmp/Timber_Industry_Spatial_1950_2025.gdb", "Timber_Industry_Spatial_20260120"
# - Timber Nonspatial: "/tmp/Timber_Nonspatial_2021_2025.gdb", "Timber_Nonspatial_20260120"
# - PFIRS:  "/tmp/PFIRS_2010_2025.gdb", "PFIRS_enriched_20260120"

# Set your dataset path and layer name here:
enriched_gdb_path = "/tmp/IFPRS_2023_2025.gdb"
enriched_layer_name = "IFPRS_enriched_20260120"

# Dataset name for display purposes
dataset_name = "IFPRS"  # Change to match your dataset (e.g., "USFS", "BLM", "NPS", etc.)

print(f"Dataset: {dataset_name}")
print(f"GDB Path: {enriched_gdb_path}")
print(f"Layer: {enriched_layer_name}")

In [ ]:
# Install necessary libraries
! pip install geopandas matplotlib seaborn folium contextily -q

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import contextily as ctx
import numpy as np
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the enriched data from GDB
print(f"Loading layer '{enriched_layer_name}' from {enriched_gdb_path}...")

# Load the layer using geopandas
enriched_data = gpd.read_file(enriched_gdb_path, 
                      sql_dialect="OGRSQL", 
                      sql=f"SELECT *, OBJECTID FROM {enriched_layer_name}")

print(f"Successfully loaded {len(enriched_data)} records")
print(f"CRS: {enriched_data.crs}")
print(f"Data columns: {len(enriched_data.columns)}")

# Display basic info
print(f"\nFirst few rows:")
display(enriched_data[['PROJECT_NAME', 'AGENCY', 'ACTIVITY_CAT', 'BROAD_VEGETATION_TYPE', 'ACTIVITY_QUANTITY']].head())

In [ ]:
# Transform to WGS84 (lat/long) for mapping
enriched_data_wgs84 = enriched_data.to_crs('EPSG:4326')
print(f"Transformed to WGS84: {enriched_data_wgs84.crs}")

# Create centroids for point mapping (reproject to avoid warnings)
enriched_data_proj = enriched_data_wgs84.to_crs('EPSG:3857')  # Web Mercator for accurate centroids
enriched_data_proj['centroid'] = enriched_data_proj.geometry.centroid
enriched_data_proj['longitude'] = enriched_data_proj.centroid.x
enriched_data_proj['latitude'] = enriched_data_proj.centroid.y

# Transform centroids back to WGS84 for mapping
centroids_wgs84 = gpd.GeoDataFrame(enriched_data_proj, geometry='centroid', crs='EPSG:3857').to_crs('EPSG:4326')
enriched_data_wgs84['longitude'] = centroids_wgs84.geometry.x
enriched_data_wgs84['latitude'] = centroids_wgs84.geometry.y

print(f"Coordinate ranges:")
print(f"Latitude: {enriched_data_wgs84['latitude'].min():.4f} to {enriched_data_wgs84['latitude'].max():.4f}")
print(f"Longitude: {enriched_data_wgs84['longitude'].min():.4f} to {enriched_data_wgs84['longitude'].max():.4f}")

## Interactive Map with Treatment Activities

In [ ]:
# Create interactive map centered on California
center_lat = enriched_data_wgs84['latitude'].mean()
center_lon = enriched_data_wgs84['longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6, tiles='OpenStreetMap')

# Add color mapping for activity categories
activity_colors = {
    'MECH_HFR': 'red',
    'TIMB_HARV': 'brown',
    'BENEFICIAL_FIRE': 'orange', 
    'NOT_DEFINED': 'gray',
    'GRAZING': 'green',
    'TREE_PLNTING': 'darkgreen',
    'LAND_PROTEC': 'purple',
    'WATSHD_IMPRV': 'blue'
}

# Add points to the map
for idx, row in enriched_data_wgs84.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        activity_cat = row['ACTIVITY_CAT']
        color = activity_colors.get(activity_cat, 'gray')
        
        popup_text = f"""
        <b>Project:</b> {row['PROJECT_NAME']}<br>
        <b>Agency:</b> {row['AGENCY']}<br>
        <b>Activity:</b> {row['ACTIVITY_CAT']}<br>
        <b>Vegetation:</b> {row['BROAD_VEGETATION_TYPE']}<br>
        <b>Quantity:</b> {row['ACTIVITY_QUANTITY']:.1f} {row.get('ACTIVITY_UOM', '')}<br>
        <b>Status:</b> {row['ACTIVITY_STATUS']}
        """
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=folium.Popup(popup_text, max_width=300),
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)

# Add legend
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 150px; height: 180px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:12px; padding: 10px">
    <h5>Activity Types</h5>
    <i class="fa fa-circle" style="color:red"></i> MECH_HFR<br>
    <i class="fa fa-circle" style="color:brown"></i> TIMB_HARV<br>
    <i class="fa fa-circle" style="color:orange"></i> BENEFICIAL_FIRE<br>
    <i class="fa fa-circle" style="color:gray"></i> NOT_DEFINED<br>
    <i class="fa fa-circle" style="color:green"></i> GRAZING<br>
    <i class="fa fa-circle" style="color:darkgreen"></i> TREE_PLNTING<br>
    <i class="fa fa-circle" style="color:purple"></i> LAND_PROTEC<br>
    <i class="fa fa-circle" style="color:blue"></i> WATSHD_IMPRV
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

display(m)

## Activity Quantity by Agency

In [ ]:
# Calculate total activity quantity by agency
agency_quantity = enriched_data.groupby('AGENCY')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
agency_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
agency_quantity = agency_quantity.sort_values('Total Quantity', ascending=False)

print(f"Activity Quantity by Agency ({dataset_name}):")
display(agency_quantity)

# Create bar chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Total quantity bar chart
agency_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title(f'Total Activity Quantity by Agency - {dataset_name}', fontsize=14, fontweight='bold')
ax1.set_xlabel('Agency')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
agency_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='lightcoral')
ax2.set_title(f'Number of Activities by Agency - {dataset_name}', fontsize=14, fontweight='bold')
ax2.set_xlabel('Agency')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Activity Quantity by Activity Types

In [ ]:
# Calculate total activity quantity by activity category
activity_quantity = enriched_data.groupby('ACTIVITY_CAT')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
activity_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
activity_quantity = activity_quantity.sort_values('Total Quantity', ascending=False)

print(f"Activity Quantity by Activity Type ({dataset_name}):")
display(activity_quantity)

# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Total quantity bar chart
activity_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='gold')
ax1.set_title(f'Total Activity Quantity by Activity Type - {dataset_name}', fontsize=14, fontweight='bold')
ax1.set_xlabel('Activity Category')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
activity_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='lightblue')
ax2.set_title(f'Number of Activities by Type - {dataset_name}', fontsize=14, fontweight='bold')
ax2.set_xlabel('Activity Category')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

# Pie chart for number of activities
activity_quantity['Number of Activities'].plot(kind='pie', ax=ax3, autopct='%1.1f%%', startangle=90)
ax3.set_title(f'Distribution of Activities by Type - {dataset_name}', fontsize=14, fontweight='bold')
ax3.set_ylabel('')

# Average quantity bar chart
activity_quantity['Average Quantity'].plot(kind='bar', ax=ax4, color='lightgreen')
ax4.set_title(f'Average Activity Quantity by Type - {dataset_name}', fontsize=14, fontweight='bold')
ax4.set_xlabel('Activity Category')
ax4.set_ylabel('Average Quantity')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Activity Quantity by Vegetation Types

In [ ]:
# Calculate total activity quantity by vegetation type
veg_quantity = enriched_data.groupby('BROAD_VEGETATION_TYPE')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
veg_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
veg_quantity = veg_quantity.sort_values('Total Quantity', ascending=False)

print(f"Activity Quantity by Vegetation Type ({dataset_name}):")
display(veg_quantity)

# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Total quantity bar chart
veg_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='forestgreen')
ax1.set_title(f'Total Activity Quantity by Vegetation Type - {dataset_name}', fontsize=14, fontweight='bold')
ax1.set_xlabel('Vegetation Type')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
veg_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='sandybrown')
ax2.set_title(f'Number of Activities by Vegetation Type - {dataset_name}', fontsize=14, fontweight='bold')
ax2.set_xlabel('Vegetation Type')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

# Pie chart for number of activities
veg_quantity['Number of Activities'].plot(kind='pie', ax=ax3, autopct='%1.1f%%', startangle=90)
ax3.set_title(f'Distribution of Activities by Vegetation Type - {dataset_name}', fontsize=14, fontweight='bold')
ax3.set_ylabel('')

# Average quantity bar chart
veg_quantity['Average Quantity'].plot(kind='bar', ax=ax4, color='mediumpurple')
ax4.set_title(f'Average Activity Quantity by Vegetation Type - {dataset_name}', fontsize=14, fontweight='bold')
ax4.set_xlabel('Vegetation Type')
ax4.set_ylabel('Average Quantity')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Summary Statistics

In [ ]:
# Overall summary statistics
print(f"=== {dataset_name} DATA SUMMARY ===")
print(f"Total number of activities: {len(enriched_data)}")
print(f"Total activity quantity: {enriched_data['ACTIVITY_QUANTITY'].sum():.2f}")
print(f"Average activity quantity: {enriched_data['ACTIVITY_QUANTITY'].mean():.2f}")
print(f"Date range: {enriched_data['ACTIVITY_START'].min()} to {enriched_data['ACTIVITY_END'].max()}")

print(f"\n=== BY AGENCY ===")
for agency in enriched_data['AGENCY'].unique():
    if pd.notna(agency):
        agency_data = enriched_data[enriched_data['AGENCY'] == agency]
        print(f"{agency}: {len(agency_data)} activities, {agency_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")

print(f"\n=== BY ACTIVITY TYPE ===")
for activity in enriched_data['ACTIVITY_CAT'].unique():
    if pd.notna(activity):
        activity_data = enriched_data[enriched_data['ACTIVITY_CAT'] == activity]
        print(f"{activity}: {len(activity_data)} activities, {activity_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")

print(f"\n=== BY VEGETATION TYPE ===")
for veg in enriched_data['BROAD_VEGETATION_TYPE'].unique():
    if pd.notna(veg):
        veg_data = enriched_data[enriched_data['BROAD_VEGETATION_TYPE'] == veg]
        print(f"{veg}: {len(veg_data)} activities, {veg_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")